# Univariate function approximation

In this note we look at how we can use a Single Layer feedforward network to approximate functions.

This will be somewhat of a lit review as I experiment with ideas or concepts from various papers

In [ ]:
using PlotlyJS
using SLFN
f1(x::Real) = log10(4(x+0.1)) + 0.4
f2(x::Number) = (1/2*cos(3π*x)+0.5)*sin(x/2)
f3(x::Number) = x < 0.4 ? x + 0.1 : (x-0.4)^2 + 0.5
f4(x::Number) = x < 0.5 ? -4*(x^2)+1 : -0.3*sin(2*π*x)

f2′(x::Number) = (1/2)*(0.5 + 0.5*cos(9.42477796076938*x))*cos((1/2)*x) - 
                 4.71238898038469*sin((1/2)*x)*sin(9.42477796076938*x)

for f in [:f1, :f2, :f3, :f4, :f2′]
    @eval $(f)(x::AbstractArray) = map($(f), x)
end

## Smooth function approximation using Neural networks

Here I keep the notes I took while reading Ferrari, S., & Stengel, R. F. (2005). Smooth function approximation using neural networks. IEEE Transactions on Neural Networks, 16(1), 24–38. http://doi.org/10.1109/TNN.2004.836233

In [14]:
function show_approx{TN<:AbstractSLFN}(f::Function, ::Type{TN}=AlgebraicNetwork, 
                                       activation=Sigmoid(), p::Int=20, s::Int=20)
    y = linspace(0, 1, p)
    u = f(y)
    print("time to do fit:")
    @time an = TN(y, u, activation, s)
    approx = an(y)
    err = u - approx
    s1 = scatter(x=y, y=u, name="data", mode="markers", legendgroup="data",
    marker=attr(symbol="line-nw-open", size=4, color="blue")
    )
    l = Layout(xaxis_range=(0, 1))
    p1 = plot([s1,
               scatter(x=y, y=an(y), name="approx", mode="lines")], l)
    y2 = linspace(0, 1, 40)
    p2 = plot([copy(s1), 
               scatter(x=y2, y=f(y2), marker_symbol="line-ns-open", mode="markers", name="fine data"),
               scatter(x=y2, y=an(y2), name="approx", marker_color="green"),
               scatter(x=y2, y=f(y2)-an(y2), marker_color="Red", name="error")], l)
    restyle!(p2, 1, showlegend=false)
    plt = [p1; p2]
    ti = s == p ? "Exact $(TN) with $(s) neurons and training points" :
                  "Approximate $(TN) with $(s) neurons and $(p) training points"
    relayout!(plt, title=ti)
    display(plt)
    an
end

show_approx (generic function with 5 methods)

In [11]:
an_exact = show_approx(f2, AlgebraicNetwork, Sigmoid(), 20, 20)

time to do fit:  0.000760 seconds (199 allocations: 152.953 KB)


AlgebraicNetwork with
  - SLFN.Sigmoid Activation function
  - 1 input dimension(s)
  - 20 neuron(s)
  - 20 training point(s)


In [12]:
an_exact = show_approx(f2, AlgebraicNetwork, Sigmoid(), 20, 20)

time to do fit:  0.004161 seconds (1.22 k allocations: 968.844 KB)


AlgebraicNetwork with
  - SLFN.Sigmoid Activation function
  - 1 input dimension(s)
  - 20 neuron(s)
  - 20 training point(s)


#### Non-smooth functions

The method is only guaranteed to work for smooth functions. But I'm curious so I want to try it with a non-smooth one.

Let's use the infamous f4 that has proven difficult to learn for all othe other networks we've tried.

In [13]:
show_approx(f4, AlgebraicNetwork, Sigmoid(), 500, 20)

time to do fit: 

 1.229576 seconds (106.20 k allocations: 731.589 MB, 5.69% gc time)


AlgebraicNetwork with
  - SLFN.Sigmoid Activation function
  - 1 input dimension(s)
  - 20 neuron(s)
  - 500 training point(s)


Not bad!

## Extreme learning machine: Theory and applications

The next paper is quite similar to the one studied above

Huang, G.-B., Zhu, Q.-Y., & Siew, C.-K. (2006). Extreme learning machine: Theory and applications. Neurocomputing, 70(1-3), 489–501. http://doi.org/10.1016/j.neucom.2005.12.126

This paper seemed very similar to the previous one, but they relax a few assumptions:

- Use of sigmoid activation -- they just require it to be infnitely differentiable
- The W and d parts are totally randomized here. Means the S matrix is only computed once
- 

In [15]:
elm_exact = show_approx(f2, ELM, Sigmoid(), 20, 20)

time to do fit:  0.000193 seconds (52 allocations: 25.766 KB)


ELM with
  - SLFN.Sigmoid Activation function
  - 1 input dimension(s)
  - 20 neuron(s)
  - 20 training point(s)


In [ ]:
elm_exact = show_approx(f2, ELM, Sigmoid(), 400, 15)

In [17]:
show_approx(f4, ELM, Sigmoid(), 500, 15)

time to do fit:  0.000437 seconds (200 allocations: 502.641 KB)


ELM with
  - SLFN.Sigmoid Activation function
  - 1 input dimension(s)
  - 15 neuron(s)
  - 500 training point(s)


#### Summary

This method seems to work about as well as the Algebraic network for the smooth cases, but not quite as well for the discontinuous function.